# **Multiplication of matrices of size n * m and m * n.**
*It must be possible to vary n and m. Matrices of a given size are generated automatically.
The program should work correctly at least up to the input data size of 2500x2500 elements.*

*For this lab, it is necessary to compare the program with the CPU implementation in terms of execution speed, as well as to compare the results obtained element by element to confirm the correctness of the GPU-accelerated implementation. For a video card, the total time for copying input data into video memory, executing the kernel function, and copying back the resulting data is measured, but the time for allocating and freeing memory is not measured.*

# **Перемножение матриц размером n * m и m * n.**
*Должна быть возможность варьирования n и m. Матрицы заданного размера генерируются автоматически.
Программа должна корректно работать хотя бы до размера входных данных 2500x2500 элементов.*

*Для этой лабораторной нужно сравнить с ЦП-реализацией по скорости выполнения, а также сравнить поэлементно полученные результаты, чтобы подтвердить корректность GPU-ускоренной реализации. Для видеокарты замеряется суммарное время копирования входных данных в видеопамять, выполнения функции-ядра и обратного копирования результирующих данных, но не замеряется время выделения и освобождения памяти.*

In [ ]:
!pip install numba

In [ ]:
import numpy as np
import time
from numba import cuda, jit, njit
import math

In [ ]:
n = 2500
m = 2500

In [ ]:
A = np.random.randint(0, 10, size = (n,m), dtype = np.int64)
B = np.random.randint(0, 10, size = (m,n), dtype = np.int64)
C = np.zeros(shape = (n,n), dtype = np.int64)

In [ ]:
#core
@cuda.jit
def matrix_mult(M1, M2, MR):
  i, j = cuda.grid(2)
  if i < M1.shape[0] and j < M2.shape[1]:
    row = M1[i,:]
    col = M2[:,j]
    r = 0
    for z in range(0, row.shape[0]):
      r += row[z]*col[z]
    MR[i,j] = r

In [ ]:
A_CUDA = cuda.to_device(A)
B_CUDA = cuda.to_device(B)
C_CUDA = cuda.to_device(C)

In [ ]:
block_size = 32
grid_size = math.ceil(C.shape[0] / (block_size))
t1 = time.time()
matrix_mult[(grid_size, grid_size),(block_size, block_size)](A_CUDA, B_CUDA, C_CUDA)
C_LOCAL = C_CUDA.copy_to_host()
t2 = time.time()
print("Время выполнения: " + str(t2 - t1) + " с")
t1 = time.time()
C_CPU = np.matmul(A, B)
t2 = time.time()
print("Время выполнения на CPU с numpy: " + str(t2 - t1) + " с")
np.array_equal(C_LOCAL, C_CPU)


Время выполнения: 1.3214707374572754 с
Время выполнения на CPU с numpy: 46.15848159790039 с


True

In [ ]:
#CPU Numba
@njit(parallel = True)
def matrix_mult_cpu_numba(M1, M2):
  l1 = M1.shape[0]
  l2 = M2.shape[1]
  R = np.zeros(shape = (l1, l2), dtype = np.int64)
  for i in range(0, l1):
    for j in range(0, l2):
      r = M1[i,:]*M2[:,j]
      R[i,j] = np.sum(r)
  return R

In [ ]:
t1 = time.time()
C_CPU = matrix_mult_cpu_numba(A, B)
t2 = time.time()
print("Время выполнения на CPU с jit: " + str(t2-t1) + " с")
np.array_equal(C_LOCAL, C_CPU)

Время выполнения на CPU с jit: 99.46723461151123 с


True